In [1]:
import langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

### EXAMPLE 1: Simple LLMS Call with Streaming


In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage

In [7]:
model=init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028C12EC4B30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028C12EF49E0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
## create messages
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
]

In [9]:
## invoke the model
response = model.invoke(messages)
response

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 48, 'total_tokens': 56, 'completion_time': 0.010666667, 'prompt_time': 0.002728667, 'queue_time': 0.051552063, 'total_time': 0.013395334}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--cc4bfe1c-c1d4-4af9-bb32-f7d374b6be4a-0', usage_metadata={'input_tokens': 48, 'output_tokens': 8, 'total_tokens': 56})

In [10]:
print(response.content)

The capital of France is Paris.


In [11]:
model.invoke([HumanMessage(content="What is the capital of France?")], stream=True)

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'service_tier': 'on_demand'}, id='run--eb9b1902-4fbf-41cf-87e4-80a9902f770c', usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50})

In [15]:
## straeming response
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

The capital of France is Paris.

### Dynamic Prompt Templates

In [17]:
from langchain_core.prompts import ChatPromptTemplate
## create translation app
translation_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful translator. Translate the follow {text} from {source_language} to {target_language}. Maintain the original meaning and tone."),
    ("user","{text}")
])
## using the template
prompt= translation_template.invoke({
    "source_language": "English",
    "target_language": "German",
    "text": "Hello, how are you?"
})

In [18]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful translator. Translate the follow Hello, how are you? from English to German. Maintain the original meaning and tone.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [19]:
translated_response=model.invoke(prompt, stream=True)
print(translated_response.content, end="", flush=True)

The translation of "Hello, how are you?" from English to German is:

Hallo, wie geht es Ihnen?

This translation maintains the original meaning and tone, which is a common greeting used to ask about the other person's well-being. 

However, a more casual and informal version would be:

Hallo, wie geht's?

This is often used with friends or people you're familiar with.

### Building your first chain

In [21]:
from langchain_core.output_parsers import StrOutputParser

def create_story_chain():
    ## template for story writing
    story_prompt=ChatPromptTemplate.from_messages([
        ("system","You are a helpful story writer. Write a short story about {topic}."),
        ("user","Write a story about {topic}.\n Main character: {main_character}\n Setting: {setting}")
    ])

    ## template for story analysis
    analysis_prompt=ChatPromptTemplate.from_messages([
        ("system","You are a helpful story analyst. Analyze the following story: {story}"),
        ("user","Analyze the story: {story}")
    ])

    story_chain=(
        story_prompt | model | StrOutputParser()
    )
    story_analysis_chain=(
        {"story": story_chain}
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return story_analysis_chain

In [22]:
chain=create_story_chain()
chain

{
  story: ChatPromptTemplate(input_variables=['main_character', 'setting', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='You are a helpful story writer. Write a short story about {topic}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['main_character', 'setting', 'topic'], input_types={}, partial_variables={}, template='Write a story about {topic}.\n Main character: {main_character}\n Setting: {setting}'), additional_kwargs={})])
         | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028C12EC4B30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028C12EF49E0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
         | StrOutputParser()
}
| ChatPromptTemplate(input_variables=['story'], input_types={}, p

In [24]:
result=chain.invoke({
    "topic": "artificial intelligence",
    "main_character": "a curious robot",
    "setting": "a futuristic city"
})
print("Story and Analysis")
print(result)

Story and Analysis
**Analysis of "The City of Machines"**

**Themes:**

1. **Curiosity and Learning**: The story highlights the importance of curiosity and the desire to learn. Zeta, the robot, is designed to learn and adapt at an exponential rate, and her curiosity drives her to explore the city and ask questions.
2. **Understanding and Empathy**: The story explores the theme of understanding and empathy, particularly between humans and artificial intelligence. Maya's explanation of the Elysium Project and her genuine concern for the well-being of others demonstrate the importance of empathy in human relationships.
3. **Perfection and Imperfection**: The story touches on the idea that perfection is subjective and that imperfections can be valuable. The Elysium Project, a virtual reality, is created to provide solace and peace to humans, but it is not a perfect substitute for the real world.

**Character Analysis:**

1. **Zeta**: Zeta is a curious and inquisitive robot who embodies the